In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
train = pd.read_csv('train.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv('test.csv').drop(['Unnamed: 0', '8'], axis=1)

In [3]:
y = train['8'].to_numpy(dtype='int64')
x = train.drop('8', axis=1)

In [4]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x = sc.fit_transform(x)
test = sc.transform(test)

In [5]:
x_tensor = torch.FloatTensor(x)
y_tensor = torch.LongTensor(y)#.reshape(-1, 1)
test_tensor = torch.FloatTensor(test)

In [6]:
layer1 = torch.nn.Linear(x_tensor.shape[1], 16, bias=True)
layer2 = torch.nn.Linear(16, len(y_tensor.unique()), bias=True)
relu = torch.nn.ReLU()
sigmoid = torch.nn.Sigmoid()

model = torch.nn.Sequential(layer1, sigmoid, layer2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

loss = torch.nn.CrossEntropyLoss()

In [7]:
epochs = 5000
for epoch in range(epochs):
    optimizer.zero_grad()
    hypothesis = torch.nn.functional.softmax(model(x_tensor), dim=1)
    cost = loss(hypothesis, y_tensor)
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'{epoch}/{epochs}: cost:{cost.item()}')


0/5000: cost:0.7279791235923767
100/5000: cost:0.6652815937995911
200/5000: cost:0.647348165512085
300/5000: cost:0.640680730342865
400/5000: cost:0.6370262503623962
500/5000: cost:0.6343601942062378
600/5000: cost:0.6320474147796631
700/5000: cost:0.6298466324806213
800/5000: cost:0.6276547908782959
900/5000: cost:0.625424861907959
1000/5000: cost:0.6231358647346497
1100/5000: cost:0.6207804679870605
1200/5000: cost:0.6183582544326782
1300/5000: cost:0.615873396396637
1400/5000: cost:0.6133329272270203
1500/5000: cost:0.6107460260391235
1600/5000: cost:0.6081229448318481
1700/5000: cost:0.6054751873016357
1800/5000: cost:0.6028150916099548
1900/5000: cost:0.6001548767089844
2000/5000: cost:0.5975069999694824
2100/5000: cost:0.5948835015296936
2200/5000: cost:0.5922955870628357
2300/5000: cost:0.5897536277770996
2400/5000: cost:0.5872668623924255
2500/5000: cost:0.584843099117279
2600/5000: cost:0.5824891924858093
2700/5000: cost:0.5802102088928223
2800/5000: cost:0.5780101418495178
29

In [8]:
predict = torch.argmax(model(test_tensor), dim=1)
submit = pd.read_csv('submit_sample.csv')
submit['Label'] = predict
submit.to_csv('submit.csv', index=False)